# Project Splash

Install needed Python libraries

In [21]:
!pip3 install numpy
!pip3 install pyserial
!pip3 install matplotlib
!pip3 install datetime

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.
  Using cached email-4.0.2.tar.gz (1.2 MB)
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/q7/zpdmv05n4sldh5kzbfqr7gp40000gn/T/pip-install-uex4omln/email_9a7511d55dd74b199d694e1a2db90ce5/setup.py'"'"'; __file__='"'"'/private/var/folders/q7/zpdmv05n4sldh5kzbfqr7gp40000gn/T/pip-install-uex4omln/email_9a7511d55dd74b199d694e1a2db90ce5/setup.py'"'"';f = getattr(tokenize, '"'"'ope

     |████████████████████████████████| 1.1 MB 18.1 MB/s            
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/q7/zpdmv05n4sldh5kzbfqr7gp40000gn/T/pip-install-uex4omln/email_2a7e9eca92824e19889d71c6152c1602/setup.py'"'"'; __file__='"'"'/private/var/folders/q7/zpdmv05n4sldh5kzbfqr7gp40000gn/T/pip-install-uex4omln/email_2a7e9eca92824e19889d71c6152c1602/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/q7/zpdmv05n4sldh5kzbfqr7gp40000gn/T/pip-pip-egg-info-c4ay40e5
       cwd: /private/var/folders/q7/zpdmv05n4sldh5kzbfqr7gp400

Import necessary libraries

In [3]:
import serial
import numpy as np
import matplotlib.pyplot as plt

import smtplib, ssl
from email.mime.base import MIMEBase 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText
from email import encoders
from datetime import datetime



N = 20  # number of samples to collect
i=0
x=np.zeros(N)
y=np.zeros(N)
i=0

low_count = 0
up_count = 0

dist_thresh = 800
inch_dist_thresh = dist_thresh/148
formatted_thresh = "{:.2f}".format(inch_dist_thresh)

arduino = serial.Serial(port='/dev/cu.usbmodem142201', baudrate=9600, timeout=.1)
arduino.close()
arduino.open()

while i < N:
    data = arduino.readline()
    if data:
        tokens = data.decode().split(',')
        y[i] = float(tokens[0])
        x[i] = float(tokens[1])
        i += 1
        distance = float(tokens[0])
        print(distance)
        
        if distance < dist_thresh:
            up_count+=1
            print("up_count")
        if distance >= dist_thresh:
            low_count+=1
            print("low_count added one")
        if low_count>0 and up_count>3:
            low_count=0
            print("low_count reset")
        if low_count==2:
            up_count=0
            print("up_count reset")
        print(" . . . ")
        print("low_count = " + str(low_count))
        print("up_count = " + str(up_count))
        if low_count==3:  
            print('firing email, son')
            port=465
            context = ssl.create_default_context()
            EMAIL_ADDRESS = 'coverletterwriter@gmail.com'
            EMAIL_PASSWORD = 'pythonemail'
            now = datetime.now()
            datetime_subject_line = now.strftime("%m/%d/%Y @ %H:%M")
            formatted_dist = "{:.2f}".format(distance/148)
            
            msg = MIMEMultipart()
            msg['Subject'] = 'Splash Device Notification - ' + datetime_subject_line
            subject = 'Splash Device Notification - ' + datetime_subject_line
            msg['From'] = EMAIL_ADDRESS
            from_txt = EMAIL_ADDRESS
            msg['To'] = ['whitfd18@wfu.edu', 'frank.d.whitworth.4@gmail.com', 'coverletterwriter@gmail.com', 'aspiers10@gmail.com']
            to_txt = ['whitfd18@wfu.edu', 'frank.d.whitworth.4@gmail.com', 'coverletterwriter@gmail.com', 'aspiers10@gmail.com']
            body_txt = """Customer,

    Your plants have crossed over your distance threshold of """ + str(formatted_thresh) + """.

    Your Splash device is currently reading """ + str(formatted_dist) + """ inches. 
    
    Please water your plant! 
    
    Have a great day.

            -Splash Team
            """
            
            email_txt = """\
    From: %s
    To: %s
    Subject: %s
    
    %s
    """ % (from_txt, ", ".join(to_txt), subject, body_txt)

    
            msg.attach(MIMEText(body_txt))

            # msg = msg.as_string()

            try:
                server = smtplib.SMTP('smtp.gmail.com:587')
                server.ehlo()
                server.starttls()
                print('started tls, big dog')
                server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
                print("logged in, boi")
                server.sendmail(EMAIL_ADDRESS, to_txt, email_txt)
                server.quit()
                print("Email sent successfully")
            except:
                print("Email could not be sent...")
                exit()
            exit()
            up_count = 0
            low_count = 0
        

arduino.close()

# print(y)
# print(x)


# plt.plot(x, y)
# plt.show


7549.0
low_count added one
 . . . 
low_count = 1
up_count = 0
7544.0
low_count added one
up_count reset
 . . . 
low_count = 2
up_count = 0
7571.0
low_count added one
 . . . 
low_count = 3
up_count = 0
firing email, son
started tls, big dog
logged in, boi
Email sent successfully
7550.0
low_count added one
 . . . 
low_count = 1
up_count = 0
7538.0
low_count added one
up_count reset
 . . . 
low_count = 2
up_count = 0
7544.0
low_count added one
 . . . 
low_count = 3
up_count = 0
firing email, son
started tls, big dog
logged in, boi
Email sent successfully
7547.0
low_count added one
 . . . 
low_count = 1
up_count = 0
7558.0
low_count added one
up_count reset
 . . . 
low_count = 2
up_count = 0
7539.0
low_count added one
 . . . 
low_count = 3
up_count = 0
firing email, son
started tls, big dog
logged in, boi
Email sent successfully
325.0
up_count
 . . . 
low_count = 0
up_count = 1


KeyboardInterrupt: 